In [7]:
# Input data
import pandas as pd

df = pd.read_csv('D:/p/Iowa_Liquor_Sales.csv')

C:\Users\user\AppData\Local\Temp\ipykernel_30444\4143939232.py:4: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('D:/p/Iowa_Liquor_Sales.csv')


In [3]:
# Parse the date column using a specific date format
df['Date'] = pd.to_datetime(df['Date'], format='%m/%d/%Y')

# Filter data between the years 2012 and 2014
start_date = '2012-01-01'
end_date = '2014-12-31'

# Filter the dataframe
f_df = df[(df['Date'] >= start_date) & (df['Date'] <= end_date)]
print(f_df)

        Invoice/Item Number       Date  Store Number  \
100            S10699400047 2013-02-20          4794   
102            S08792800017 2012-11-06          2459   
103            S04854100048 2012-04-03          2487   
104            S13206600042 2013-07-08          3908   
106            S08900500035 2012-11-13          4509   
...                     ...        ...           ...   
8197498        S20177600011 2014-07-17          3666   
8197500        S13373400046 2013-07-16          4297   
8197501        S09930800322 2013-01-08          2614   
8197502        S06961800244 2012-08-06          2500   
8197504        S04191200005 2012-02-21          3842   

                                   Store Name                      Address  \
100      Smokin' Joe's #17 Tobacco and Liquor       110 S ROOSEVELT HWY 61   
102                            Reinhart Foods          200 STATE PO BOX 98   
103                      Anamosa Family Foods                402 EAST MAIN   
104         Cou

In [4]:
# Pick out variable "Bottles Sold" to make analysis
selected_columns = ['Item Description', 'Bottles Sold']
df_selected = f_df[selected_columns]
print(df_selected)

                       Item Description  Bottles Sold
100                             Absente             4
102                    Cruzan Mango Rum             3
103           Arrow Peppermint Schnapps             3
104           Uv Blue (raspberry) Vodka            12
106                       Kinky Liqueur            12
...                                 ...           ...
8197498             Cedar Ridge Bourbon             6
8197500                   Tanqueray Gin             2
8197501  Arrow Peppermint Schnapps Mini             1
8197502                Cabo Wabo Blanco             6
8197504   Jack Daniels Old #7 Black Lbl            12

[6243618 rows x 2 columns]


In [4]:
# I only have the image of first 912 liquors' image. Therefore, extract first 912 liquors sales data
from scipy import stats
grouped_df = df_selected.groupby('Item Description')['Bottles Sold'].sum().reset_index()
sorted_df = grouped_df.sort_values(by='Bottles Sold', ascending=False).reset_index(drop=True)
# print(sorted_df)
# sorted_df.to_csv('D:/p/Iowa_Liquor_Sales_Sorted.csv', index=False)
combined_df = pd.concat([sorted_df.head(10), sorted_df.tail(10)], ignore_index=True)
# combined_df.to_csv('D:/p/Iowa_Liquor_Sales_f.csv', index=False)
# print(combined_df)
# print(sorted_df)
combined_df_t = sorted_df.head(912)


In [11]:
# Standarlize my liquor database to make the analysis more clear and apparent
from scipy import stats
import numpy as np

# Calculate Z-scores for 'Bottles Sold'
combined_df_t.loc[:, 'Z-score'] = np.abs(stats.zscore(combined_df_t['Bottles Sold']))

# Identify outliers (Z-score > 3)
outliers = combined_df_t[combined_df_t['Z-score'] > 3]

# Print or display outliers
# print(outliers)

# Optionally, remove outliers
cleaned_df = combined_df_t[combined_df_t['Z-score'] <= 3]


In [6]:
# Imput images database and add them into the dataframe with liquors' sales
import os
import warnings

# Define the image folder path
image_folder = r'D:\p\liquor_images'

# Generate a list of image filenames
image_filenames = [f'l{i}.png' for i in range(18, 913)]

# Create a list of full image paths
image_paths = [os.path.join(image_folder, filename) for filename in image_filenames]

# Add the image paths to the 'cleaned_df' dataframe
cleaned_df['image_paths'] = image_paths
cleaned_df.to_csv('D:/p/Iowa_Liquor_Sales_f.csv', index=False)
warnings.filterwarnings('ignore')

In [7]:
# Using pre training model VGG16 to extract features of liquors' images
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input
import sys
import contextlib

# Suppresses TensorFlow info and warnings, shows only errors
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

# Load pre-trained VGG16 model (without the top classification layer)
base_model = VGG16(weights='imagenet', include_top=False, pooling='avg')
model = Model(inputs=base_model.input, outputs=base_model.output)

# Function to suppress stdout (to hide the output during feature extraction)
@contextlib.contextmanager
def suppress_stdout():
    with open(os.devnull, 'w') as devnull:
        old_stdout = sys.stdout
        sys.stdout = devnull
        try:
            yield
        finally:
            sys.stdout = old_stdout
            
def extract_features(img_path):
    if os.path.exists(img_path):
        # Load image and resize to (224, 224) as required by VGG16
        img = image.load_img(img_path, target_size=(224, 224))
        img_data = image.img_to_array(img)
        img_data = np.expand_dims(img_data, axis=0)
        img_data = preprocess_input(img_data)  # Preprocess the image as expected by VGG16
        
        # Extract features using VGG16 model without progress bar (verbose=0)
        features = model.predict(img_data, verbose=0)
        return features.flatten()  # Flatten the features for further use
    else:
        print(f"File not found: {img_path}")
        return None
cleaned_df['image_paths'] = cleaned_df['image_paths'].str.replace('.png', '.jpg')
# Apply feature extraction to your dataframe (assuming 'cleaned_df' has a column 'image_paths')
cleaned_df['features'] = cleaned_df['image_paths'].apply(extract_features)

# Optional: check extracted features
# print(cleaned_df.head())

In [8]:
# Extract features and sales data into a new dataset
features = np.array(cleaned_df['features'].tolist())
sales = cleaned_df['Bottles Sold'].values
sales_log = np.log1p(sales)

# Check the shape of features and sales data
print(features.shape)
print(sales_log.shape)

(895, 512)
(895,)


In [9]:
# Using CNN to train and predict liquors sales by their images
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
# Scale the features
scaler = StandardScaler()
features_scaled = scaler.fit_transform(features)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features_scaled, sales_log, test_size=0.2, random_state=42)

model = Sequential([
    Dense(128, input_dim=features_scaled.shape[1], activation='relu', kernel_regularizer=l2(0.001)),
    Dense(64, activation='relu', kernel_regularizer=l2(0.001)),
    Dense(32, activation='relu', kernel_regularizer=l2(0.001)),
    Dense(1)
])

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.0001), loss='mean_squared_error')

# Train the model
history = model.fit(X_train, y_train, epochs=1000, batch_size=16, validation_split=0.2, verbose=0)

# Evaluate the model
y_pred = model.predict(X_test)

# Calculate Mean Squared Error
mse = mean_squared_error(y_test, y_pred)
print(f'Mean Squared Error: {mse}')

# Print a few predictions
results_df = pd.DataFrame({
    'Actual': y_test,
    'Predicted': y_pred.flatten()  # Flatten to match dimensions
})
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'  # Suppresses TensorFlow info and warnings, shows only errors
print(results_df.head(5))


6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step 
Mean Squared Error: 3.6451971569558914
      Actual  Predicted
0   8.718827   8.860093
1   9.519148  10.278123
2   9.268137   9.583838
3   8.689633   7.243011
4  12.321858   9.143518


In [10]:
# Check the predicting effect
from sklearn.metrics import mean_absolute_error, r2_score, mean_squared_error

# Calculate MSE, MAE, and R-squared
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

# Print the evaluation metrics
print(f'Mean Squared Error (MSE): {mse}')
print(f'Mean Absolute Error (MAE): {mae}')


Mean Squared Error (MSE): 3.6451971569558914
Mean Absolute Error (MAE): 1.4924331261723058
